# **EDA of 400 samples from amazon_del**  
Here I am doing an EDA on this sample in order to prepare them for a prediction model of Delivery time.  

## **Packages**

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math
from geopy.geocoders import Nominatim
import haversine as hs
from haversine import Unit
from modules_amazon import Location
import pickle

In [13]:
with open("amazon_sample400.pkl", "rb") as f:
    amazon_sample400 = pickle.load(f)

# **Checking** 

In [14]:
print(amazon_sample400.info())

<class 'pandas.core.frame.DataFrame'>
Index: 398 entries, 6607 to 18798
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   Order_ID                398 non-null    object         
 1   Agent_Age               398 non-null    int64          
 2   Agent_Rating            398 non-null    float64        
 3   Store_Latitude          398 non-null    float64        
 4   Store_Longitude         398 non-null    float64        
 5   Drop_Latitude           398 non-null    float64        
 6   Drop_Longitude          398 non-null    float64        
 7   Order_Date              398 non-null    datetime64[ns] 
 8   Order_Time              398 non-null    object         
 9   Pickup_Time             398 non-null    datetime64[ns] 
 10  Weather                 398 non-null    object         
 11  Traffic                 398 non-null    category       
 12  Vehicle                 398 non-null

In [15]:
amazon_sample400.drop(["Store_Latitude","Store_Longitude"], axis=1, inplace=True)

## **Cities**  
I have previously define the cities using the modules_amazon. However, in the EDA of the amazon_delivery I found 3180 rows with null coordinates of Store (represents around 7% of the all dataset).  
Here, I will check the cities that loc.city() had found. Especially where the corrdinates of the Store were null.

In [16]:
print(amazon_sample400["City"].isna().sum())
no_city = amazon_sample400[amazon_sample400["City"].isna()]
no_city

1


,Order_ID,Agent_Age,Agent_Rating,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,Distance_Store_Drop_km,abs_Store_Latitude,abs_Store_Longitude,duration,duration_min,City
14998,nqmv986411624,37,4.6,18.556216,73.882527,2022-03-07,13:50:00,2024-12-05 14:00:00,Sunny,High,motorcycle,Metropolitan,130,Snacks,6.129158,18.516216,73.842527,0 days 00:10:00,10.0,None


In [20]:
loc = Location(18.5,73.8)
city = loc.city()

In [22]:
print(city)
print(loc.country())

Pune City
India


In [26]:
Pune_City = amazon_sample400[amazon_sample400["City"] == city]
print(Pune_City.info())
print(Pune_City[["abs_Store_Latitude","abs_Store_Longitude"]].describe())

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 3400 to 18798
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   Order_ID                34 non-null     object         
 1   Agent_Age               34 non-null     int64          
 2   Agent_Rating            34 non-null     float64        
 3   Drop_Latitude           34 non-null     float64        
 4   Drop_Longitude          34 non-null     float64        
 5   Order_Date              34 non-null     datetime64[ns] 
 6   Order_Time              34 non-null     object         
 7   Pickup_Time             34 non-null     datetime64[ns] 
 8   Weather                 34 non-null     object         
 9   Traffic                 34 non-null     category       
 10  Vehicle                 34 non-null     category       
 11  Area                    34 non-null     category       
 12  Delivery_Time           34 non-null  

In [27]:
amazon_sample400["City"] = amazon_sample400["City"].fillna("Pune City")

I apparently don't have any null Store coordinates values in my sample.  
The only null City value was one that was found to be in Pune City. During the calculation (which requires internet access) this row might have been skipped.  

In [29]:
print(amazon_sample400["City"].unique())

['' 'Kolkata' 'Surat' 'Bengaluru' 'Prayagraj' 'Mysuru' 'Mumbai Suburban'
 'Chhatrapati Sambhaji Nagar' 'Hyderabad' 'Kanpur' 'Pune City' 'Chennai'
 'Ranchi' 'Ludhiana' 'Jaipur' 'Ernakulam' 'Bhopal' 'Indore' 'Mumbai City'
 'Agra' 'Vadodara' 'Coimbatore' 'Dehradun' 'Thane' 'Panaji']


In [31]:
empty_cities = amazon_sample400[amazon_sample400["City"] == ""]
print(empty_cities[["abs_Store_Latitude","abs_Store_Longitude"]].describe())

       abs_Store_Latitude  abs_Store_Longitude
count          103.000000           103.000000
mean            11.804214            55.019124
std              8.357637            33.846224
min              0.000000             0.000000
25%              0.000000             0.000000
50%             12.933284            73.165012
75%             21.149669            76.997008
max             30.885814            80.249982


In [32]:
print(empty_cities["Area"].value_counts())

Area
Metropolitan    85
Urban           16
Other            2
Semi-Urban       0
Name: count, dtype: int64


I won't remove them. I have a total of 103 null Store coordinates. However, I have the Area information which can be useful when comparing the traffic.